In [27]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import panel as pn

import hvplot.pandas

# Initial Inspection of Data:

In [28]:
df = pd.read_csv("ghg-emissions.csv")
x = 250

for col in df.columns:
    print(col)
    print(df[col].dtype)
    if len(df[col].unique()) < x:
        print(df[col].unique())
        print("________________________")
    else:
        print("[N/A]")

CSDUID
int64
[4801003 4801006 4802012 4802021 4802022 4803014 4804004 4804006 4804011
 4805012 4805018 4805049 4806001 4806006 4806014 4806016 4806019 4806021
 4806026 4806029 4806032 4806034 4806036 4807026 4807036 4807049 4808001
 4808011 4808022 4808024 4808031 4808044 4809010 4809015 4810019 4810039
 4811001 4811021 4811024 4811031 4811032 4811034 4811038 4811045 4811052
 4811056 4811061 4811064 4811065 4812002 4812004 4812009 4813029 4813030
 4813031 4813046 4814003 4814019 4814024 4815007 4815015 4816037 4817024
 4817026 4817029 4817031 4817076 4817093 4817095 4817097 4818002 4818005
 4818015 4818018 4819006 4819011 4819012 4819014 4819038 4819058 4819059
 4819068 4819071]
________________________
CSD Name
object
['Cypress County' 'Medicine Hat' 'Lethbridge' 'Taber, M.D.' 'Taber'
 'Pincher Creek' 'Special Area No. 2' 'Empress' 'Hanna' 'Wheatland County'
 'Strathmore' 'Trochu' 'Foothills No. 31' 'High River' 'Rocky View County'
 'Calgary' 'Cochrane' 'Airdrie' 'Crossfield' 'Carstai

In [29]:
df["Value"] = df["Value"].str.replace(",", "")
df["Value"] = pd.to_numeric(df["Value"])
idf = df.interactive() 

In [30]:
mun_list = []
for mun in df["CSD Name"].unique():
    if df.loc[df["CSD Name"] == mun]["Year"].nunique() >= 3:
        mun_list.append(mun)

autocomplete = pn.widgets.AutocompleteInput(
    name='Municipalty', options=mun_list,
    case_sensitive=False, search_strategy='includes',
    placeholder='CSD Name',
    value = "Red Deer County")

year_slider1 = pn.widgets.IntSlider(name = 'Year', start = 2004, end = 2014, step = 1, value = 2004)
year_slider2 = pn.widgets.IntSlider(name = 'Year', start = 2004, end = 2014, step = 1, value = 2004)

etype1 = pn.widgets.RadioButtonGroup(
    name='Emission Type', 
    options= list(df["Emission Type"].unique())
)
etype2 = pn.widgets.RadioButtonGroup(
    name='Emission Type', 
    options= list(df["Emission Type"].unique())
)
etype3 = pn.widgets.RadioButtonGroup(
    name='Emission Type', 
    options= list(df["Emission Type"].unique())
)
etype4 = pn.widgets.RadioButtonGroup(
    name='Emission Type', 
    options= list(df["Emission Type"].unique())
)

# Data Pipelines and Plots

In [31]:
plot_data = (
    idf[(idf["Emission Type"] == etype4) & (idf["CSD Name"] == autocomplete)].groupby(["CSD Name", "Year"])["Value"].mean().to_frame().reset_index().sort_values(by = "Year").reset_index(drop = True)
)
plot = plot_data.hvplot(x = "Year", by = "CSD Name", y = "Value", title = "Emission Values Over Time")
hist_data = (
    idf[(idf["Year"] == year_slider1) & (idf["Emission Type"] == etype1)]['Value']
)

hist = hist_data.hvplot.hist(title = "Emission Values")
boxplot_data = (
    idf[idf["Emission Type"] == etype2][["Year", "Value"]]
)

boxplot_data_lim = (
    idf[(idf["Emission Type"] == etype3) & 
        (idf[(idf["Emission Type"] == etype3)]["Value"] <
        (idf[(idf["Emission Type"] == etype3)].groupby(["Year"])["Value"].quantile(0.9).max())
        )][["Year", "Value"]]
)

boxplot = boxplot_data_lim.hvplot.box(by = "Year", 
                        title = "GHG Emission Values over the Years (Zoomed in)".format(etype3),
                        invert = True)

boxplot2 = boxplot_data.hvplot.box(by = "Year", 
                        title = "GHG Emission Values over the Years".format(etype2),
                        invert = True)

# Panel Dashboard

In [32]:
mun_str = ""
for mun in mun_list:
    mun_str = mun_str + mun + "\n###"

template = pn.template.FastListTemplate(
    title='Greenhouse Gas Emission of Municipalities - Government of Alberta', 
    sidebar = [pn.pane.Markdown("### Amount of CO2-equivalent emissions from facilities emitting over 10kt, \
    as required by Canada's Greenhouse Gas Reporting Program, by year and municipality or municipal district. \
    CO2 equivalent is a term for describing different green house gases (like methane, nitrous oxide, HFCs, etc.) in a common unit. \
    The CO2 equivalent would signify the amount of CO2 required to have a equivalent greenhouse effect."),
              pn.pane.Markdown("## List of Municipalities with Data of Emissions over time:\n###{}".format(mun_str))], 
    main=[pn.Row(pn.Column(hist, margin = (0, 25), sizing_mode="stretch_height"), 
                 pn.Column(plot, margin = (0, 25), sizing_mode="stretch_height")), 
          pn.Row(pn.Column(boxplot, margin = (0, 25)),
                 pn.Column(boxplot2, margin = (0, 25)), 
                 sizing_mode="stretch_both")],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

template.show()

Launching server at http://localhost:49894
